# Indexing Process

This notebooks is to index the documents to the vector database

In [4]:
from dotenv import load_dotenv 

load_dotenv() 

True

In [16]:
import chromadb

chroma_client = chromadb.PersistentClient(path="chromadb")

In [7]:
import os
from supabase import create_client, Client

supabase_url = os.getenv("SUPABASE_URL")
supabase_key = os.getenv("SUPABASE_KEY")
supabase_client = create_client(supabase_url, supabase_key)

## Article

In [26]:
import pandas as pd

articles = supabase_client.table("article").select("*").execute().model_dump()['data']
df_article = pd.DataFrame(articles)
df_article

,article_id,title,category,short_desc,created_date,author,body
0,1,Mastering Productivity: A Beginner's Guide,Productivity,A beginner's guide to enhancing productivity b...,2024-05-12,None,In a world brimming with distractions and dema...
1,2,"Balancing Work, Hobbies, and Family: The Ultim...",Work-Life Balance,Exploring practical strategies for achieving a...,2024-04-29,None,"In the modern whirlwind of responsibilities, f..."
2,4,"The Art of Harmonizing Career, Personal Life, ...",Work-Life Balance,"discusses techniques for balancing career, per...",2024-05-25,None,"In the intricate tapestry of life, mastering t..."
3,5,Achieving Peak Productivity: Essential Strateg...,Productivity,"Strategies for boosting productivity, such as ...",2024-05-26,None,In today's fast-paced and competitive landscap...
4,3,Current Market Outlook for China: Finance and ...,Finance,China's finance and banking sectors in 2024 pr...,2024-05-25,Patricia Kim,"As 2024 progresses, China's financial and bank..."


In [18]:
# Indexing to ChromaDB

try:
    article_collection = chroma_client.get_collection("article")
except:
    article_collection = chroma_client.create_collection("article")

In [29]:
def create_article_sentence(article):
    return f"{article['title']}. {article['short_desc']}"

In [32]:
article_collection.add(
    ids=[str(article['article_id']) for article in articles],
    documents=[create_article_sentence(article) for article in articles],
    metadatas=[
        {
            "category": article['category'],
            "created_date": article['created_date'],
        } for article in articles        
    ]
)